# 03 - Interactive Viz

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Please read the notebook from [this URL](http://nbviewer.jupyter.org/github/nunomota/ada2017-hw/blob/master/Homework_03/Homework_03.ipynb).

### Content:
* [Task 1](#t1)
* [Task 2](#t2)
* [Task 3](#t3)
* [Task 4](#t4)

# Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import folium
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline

# We will use this to compute the harmonic mean in Task 4
from scipy import stats

# Constants definition

In [ ]:
DATA_PATH = './data/'
TOPOJSON_PATH = './topojson/'
EUROSTAT_FILE = 'lfsq_urgaed_sheets.xls'
AMSTAT_FILE = '2_1 Tasso di disoccupazione.xlsx'
ORIGIN_AGE_FILE = '1_1 Disoccupati e persone in cerca d_impiego secondo criteri vari.xlsx'
LATLONG_FILE = 'countries_latlong.csv'
 
COORDINATE_EUROPE = [54.5260, 15.2551]
COORDINATE_SWITZERLAND = [46.8182, 8.2275]

# Task 1 <a name="t1">

From the eurostat website we found a dataset that includes the European unemployment rates at a recent date. We made sure that the chosen dataset included the rates for Switzerland as well, since we would like to compare Switzerland's rate to other countries'. From the datasets including information for Switzerland we chose the one containing the quaterly unemployment rates, so that we could get the most recent data.

When downloading the dataset we decided to include both the ID and the name for each country. It might happen that some countries have a different ID or a different name in the dataset and the TopoJSON file, and by using both we can verify that all the countries are matched correctly.

#### Read European data
Therefore, we first read the `.xls` file containing the unemployment rates per country. 

In [ ]:
# Read Europe unemployment data
def read_europe_df(sheet):
    """Reads xls file containing European data, and fixes country IDs to match
    TopoJSON file.
    """
    # Read xls file
    df = pd.read_excel('{dp}{f}'.format(dp=DATA_PATH, f=EUROSTAT_FILE),
        sheetname=sheet, skiprows=[12,13,14,15,16,17], header=10, na_values=':')

    # Rename country IDs and names columns
    df.rename(columns={'GEO': 'id', 'GEO(L)/TIME': 'name'}, inplace=True)

    # We need to replace the IDs for UK and Greece so that they match TopoJSON IDs
    return df.replace('UK', 'GB').replace('EL', 'GR')

europe_df = read_europe_df('Data5')

The following is a sample of our dataframe. It includes the quaterly unemployment rates until the second quarter of 2017.

In [ ]:
europe_df.head()

#### Obtain recent unemployment rate

We need to decide the information that we will use. We are interested in the most recent data, however, considering only the most recent quarter or month can lead to conclussions that are only valid for that month or quarter. This is due to the fact that there are seasonal trends on unemployment depending on the time of the year.

As a result, we decide to average the rates for the 4 most recent quarters, that is:
* 2016, Quarter 3
* 2016, Quarter 4
* 2017, Quarter 1
* 2017, Quarter 2

In this way we have the most recent information over a year, making sure that we avoid any possible seasonal trends on unemployment.

In [ ]:
europe_df['recent'] = europe_df[['2016Q3', '2016Q4', '2017Q1', '2017Q2']].mean(axis=1)
recent_europe_df = europe_df[['id', 'recent']]
recent_europe_df.head()

#### Remove countries from TopoJSON file

We now read and modify the topoJSON file for Europe. We remove from it all the countries that we do not have information about, so that they are not plotted in the map later on.

In [ ]:
# Load topojson file for Europe
europe_topo_path = r'{tp}europe.topojson.json'.format(tp=TOPOJSON_PATH)
geo_json = json.load(open(europe_topo_path))

# We keep only the countries we have information of
geo_countries = []
for c in geo_json['objects']['europe']['geometries']:
    if c['id'] in list(europe_df['id']):
        geo_countries.append(c)

# We substitute the list of countries
geo_json['objects']['europe']['geometries'] = geo_countries
geo_data = json.dumps(geo_json)

#### Create map

Lastly, we create the map for Europe.

We chose a color scale that goes from light orange to dark red. Since red is commonly associated with *danger*, it makes sense to use a darker red for countries (and cantons in the next exercises) with the highest unemployment rates.

We split this color scale in 6 intervals, each of them representing 1/6 of the total rate range. We considered that this choice of colors for each country would be the most intuitive. By using 6 colors we can see more differences between countries, still having enough contrast between colors to easily identify them.

#### Create markers for the map

TODO: Explain a bit?

In [ ]:
DEFAULT_FONT_SIZE = 24
DEFAULT_IMG_WIDTH = 64
DEFAULT_IMG_HEIGHT = 64
DEFAULT_TEXT_OFFSET = 8
DEFAULT_FONT_COLOR = 'white'
DEFAULT_BG_COLOR = 'black'
DEFAULT_BG_ALPHA = 0.6
DEFAULT_BG_RADIUS_COMPENSATION = DEFAULT_IMG_WIDTH/20
DEFAULT_PALETTE_ICON = 'https://i.imgur.com/HJLCoHt.png'

def generate_custom_icon(percentage):
    """
    Returns a fully customized DivIcon that can be used
    as a Marker - in folium.
    """
    return DivIcon(
        icon_size=(DEFAULT_IMG_WIDTH, DEFAULT_IMG_HEIGHT),
        icon_anchor=(DEFAULT_IMG_WIDTH/2, DEFAULT_IMG_HEIGHT/2),
        html=generate_custom_icon_html(percentage)
    )
    
def generate_custom_icon_html(percentage):
    """
    Returns an inlined HTML string representing the
    customized icon, along with a centered percentage.
    """
    html_string = (
        '<div>'
            '<svg width="{1}" height="{2}">'
                '<defs>'
                    '<pattern id="image-bg" x="0" y="0" height="{2}" width="{1}" patternUnits="userSpaceOnUse">'
                        '<image width="{1}" height="{2}" xlink:href="{4}"></image>'
                    '</pattern>'
                '</defs>'
                '<circle cx="{5}" cy="{6}" r="{11}" fill="{9}" fill-opacity="{10}"/>'
                '<circle cx="{5}" cy="{6}" r="{5}" fill="url(#image-bg)"/>'
                '<text x="{5}" y="{7}" font-size="{0}" text-anchor="middle" fill="{3}">{8}%</text>'
            '</svg>'
        '</div>'
    )
    
    return html_string.format(
        DEFAULT_FONT_SIZE,
        DEFAULT_IMG_WIDTH, 
        DEFAULT_IMG_HEIGHT,   
        DEFAULT_FONT_COLOR,
        DEFAULT_PALETTE_ICON,
        DEFAULT_IMG_WIDTH/2,
        DEFAULT_IMG_HEIGHT/2,
        DEFAULT_IMG_HEIGHT/2 + DEFAULT_TEXT_OFFSET,
        round(percentage),
        DEFAULT_BG_COLOR,
        DEFAULT_BG_ALPHA,
        DEFAULT_IMG_WIDTH/2 - DEFAULT_BG_RADIUS_COMPENSATION
    )

We downloaded a CSV file containing the LatLong coordinates for every country, that we will use to set the markers with the information for each country.

In [ ]:
# Read LatLong coordinates into a dataframe
latlong_df = pd.read_csv('{dp}{f}'.format(dp=DATA_PATH, f=LATLONG_FILE))
latlong_df.columns = latlong_df.columns.map(lambda x: x.strip())

# Merge the LatLong coordinates into the already existing dataframe
recent_europe_df = recent_europe_df.merge(latlong_df, how='inner', left_on='id', right_on='country').drop(['country'], axis=1)

#### Create map

Lastly, we create the map for Europe.

We chose a color scale that goes from light orange to dark red. Since red is commonly associated with *danger*, it makes sense to use a darker red for countries (and cantons in the next exercises) with the highest unemployment rates.

We split this color scale in 6 intervals, each of them representing 1/6 of the total rate range. We considered that this choice of colors for each country would be the most intuitive. By using 6 colors we can see more differences between countries, still having enough contrast between colors to easily identify them.

In [ ]:
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Create a new map
map_europe = folium.Map(COORDINATE_EUROPE, tiles=None, zoom_start=3)

# Add tile layers
folium.TileLayer('cartodbpositron').add_to(map_europe)

# Plot total unemployment
map_europe.choropleth(geo_data=json.loads(geo_data), name='Total Population',
    data=recent_europe_df, columns=['id', 'recent'], key_on='feature.id', fill_opacity=1,
    fill_color='YlOrRd', topojson='objects.europe')

# Add Marker cluster
marker_cluster = MarkerCluster(name='Markers').add_to(map_europe)
for index, row in recent_europe_df.iterrows():
    lat = row['latitude']
    long = row['longitude']
    percent = row['recent']
    country_name = row['name']
    folium.Marker([lat, long], icon=generate_custom_icon(percent), popup=country_name).add_to(marker_cluster)

# Add LayerControl
folium.LayerControl(collapsed=False).add_to(map_europe)

You can now visualize and interact with the map!

* You can zoom in to show the markers with the information for each country. You can also click on the clusters of markers to zoom in directly.

* To see the country's name you can click on its marker.

* The markers make it easier to see which class each country is assigned to.

In [ ]:
map_europe

The unemployment rate in Switzerland is one of the lowest in Europe, as can be verified since it has been assigned the lightest color.

Looking at numbers, Swiss unemployment rate is 5% and only Iceland, Malta, the Czech Republic, Germany and Norway have a lower rate (3%-4%).

In general the countries in southern Europe have higher unemployment rates, especially in the cases of Macedonia, Greece and Spain.

# Task 2 <a name="t2">
In the amstat website we found a dataset including the unemployment rates in Switzerland for the last 12 months.

#### Read data
We read the unemployment rates for the last 12 months. In this case we create a function that returns a dataframe containing the average (*Totale*) for those months, as well as a column with the IDs for each canton. 

Therefore, the variables we read from this dataset for each canton are:
* Unemployment rate
* Unemployed
* Looking for job
* Not unemployed looking for job

In [ ]:
# Load topojson file for Europe
swiss_topo_path = r'{tp}ch-cantons.topojson.json'.format(tp=TOPOJSON_PATH)
geo_json = json.load(open(swiss_topo_path))

In [ ]:
def get_clean_swiss_df(file, headers, dictionary, ids):
    """Reads xls file containing Swiss data.
    Translates variable names to English and gets the 'Total' column
    for each variable.
    """
    # Read xls file containing Swiss unemployment data
    df = pd.read_excel('{dp}{f}'.format(dp=DATA_PATH, f=file), header=headers)
    df.drop(('Metriche', 'mese'), axis=1, inplace=True)

    # We will create a dataframe in English with the variables we need
    clean_df = pd.DataFrame()

    # For each variable, we get the 'Total' column
    variables_list = df.columns.get_level_values(0).unique()
    for v in variables_list:
        clean_df[italian_english_variables[v]] = df[(v, 'Totale')]

    clean_df.drop('Totale', inplace=True)

    # Add 'id' column to the dataframe
    clean_df['id'] = ids
    
    return clean_df

The IDs were directly taken from the TopoJSON file, since we manually verified that the cantons are in the same order in both the TopoJSON file and the data files we obtained from amstat. We translate the names of the variables to English using a manually defined dictionary.

In [ ]:
# Manually define dictionary with translations to English
italian_english_variables = {
    "Tasso di disoccupazione": 'Unemployment rate',
    "Disoccupati registrati": 'Unemployed',
    "Persone in cerca d'impiego": 'Looking for job',
    "Persone in cerca d'impiego non disoccupate": 'Not unemployed looking for job',
    'stranieri': 'Foreigners',
    'svizzeri': 'Swiss'
}

In [ ]:
# Get the 'id' for each canton directly from the JSON file
cantons_id = []
for c in geo_json['objects']['cantons']['geometries']:
    cantons_id.append(c['id'])

switzerland_df = get_clean_swiss_df(file=AMSTAT_FILE, headers=[2,3],
    dictionary=italian_english_variables, ids=cantons_id)

The following is a sample of the resulting dataframe:

In [ ]:
switzerland_df.set_index('id').head()

#### Add data on people looking for a job
The Swiss Confederation defines the `Unemployment rate` as the number of unemployed registered, divided by the size of the active population, multiplied by 100 ([reference](https://www.amstat.ch/v2/definition.jsp?lang=fr)).

We will now include the `Looking for job rate`. We will compute it as the size of the respective population, divided by the size of the active population and multiplied by 100. We will then plot both rates to see if there are differences.

We do not compute and plot the `Not unemployed looking for job` rate (i.e. people who already have a job and are looking for a new one), since it is equal to substracting the `Unemployment rate` from the `Looking for job rate`.

In [ ]:
switzerland_df['Looking for job rate'] = (switzerland_df['Unemployment rate'] *
    switzerland_df['Looking for job'] / switzerland_df['Unemployed'])

switzerland_df.drop(['Unemployed', 'Looking for job',
    'Not unemployed looking for job'], axis=1).set_index('id').head()

#### Create map <a name="cm">

We now create the map showing the unemployment rate in Swiss cantons. We use the same color range and intervals as for the Europe map.

In [ ]:
# Create a new map
map_swiss = folium.Map(COORDINATE_SWITZERLAND,
    tiles='cartodbpositron', zoom_start=8)

# Plot people looking for a job
map_swiss.choropleth(geo_data=geo_json, name='Looking for job',
    data=switzerland_df, columns=['id', 'Looking for job rate'], key_on='feature.id', fill_opacity=1,
    fill_color='YlOrRd', topojson='objects.cantons')

map_swiss.choropleth(geo_data=geo_json, name='Unemployed',
    data=switzerland_df, columns=['id', 'Unemployment rate'], key_on='feature.id', fill_opacity=1,
    fill_color='YlOrRd', topojson='objects.cantons')

# Add markers with information on unemployment per canton
#feature_group = FeatureGroup(name='Some icons')
#Marker(location=[45.3288, -121.6625],
       #popup='Mt. Hood Meadows').add_to(feature_group)

#Marker(location=[45.3311, -121.7113],
       #popup='Timberline Lodge').add_to(feature_group)

#feature_group.add_to(m)

# Add LayerControl
folium.LayerControl(collapsed=False).add_to(map_swiss)

You can now interact with the map!
* You can compare the unemployment rate and the rate of people looking for a job by checking one or the other in the box at the right.
* Notice the two color scales on top: the one in the left corresponds to people looking for a job, the one in the right corresponds to only those unemployed.
* Since the color scales for both rates are different, from this figure we can't directly compare the two rates to each other. However, this allows us to compare how the cantons are doing regarding both rates.

In [ ]:
map_swiss

TODO: Comment this... ?

# Task 3 <a name="t3">
We use the amstat website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between Swiss and foreign workers.

The Economic Secretary (SECO) releases a monthly report on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for foreign (5.1%) and Swiss (2.2%) workers.

## 3.1 Data per origin

#### Read data
We now define a function to compute the average unemployment data for a dataframe containing the data for several months.

In [ ]:
def get_average_swiss_df(df, dictionary, ids):
    """Reads an xls file containing Swiss data for several months but
    without a 'Total' column. Computes for each variable the yearly average
    from the monthly values.
    """
    # We will create a dataframe in English with the variables we need
    clean_df = pd.DataFrame()

    # For each variable, we get an average column
    variables_list = df.columns.get_level_values(0).unique()
    for v in variables_list:
        clean_df[italian_english_variables[v]] = df[v].mean(axis=1)
    
    # Drop 'Totale' row
    clean_df.drop('Totale', inplace=True)
    
    # Add 'id' column to the dataframe
    clean_df['id'] = ids
    
    # Drop variables we will not use
    clean_df.drop(italian_english_variables["Persone in cerca d'impiego"],
        axis=1, inplace=True)
    clean_df.drop(italian_english_variables["Persone in cerca d'impiego non disoccupate"],
        axis=1, inplace=True)
    
    return clean_df

We read the file containing the unemployment data for both foreigners and Swiss.

Then, we create a dictionary containing two dataframes: one for foreigners and another one for Swiss. We apply the function to each of the dataframes to obtain the average per month.

In [ ]:
# Read xls file containing unemployment data per origin
origin_df = pd.read_excel('{dp}{f}'.format(dp=DATA_PATH, f="foreign swiss {df}".format(df=AMSTAT_FILE)),
    header=[2,3,4])
origin_df.drop(('Nazionalità', 'Metriche', 'mese'), axis=1, inplace=True)

# We create a dictionary of dataframes
origin_df_dict = {}

# For each origin we get the corresponding dataframe
origins_list = origin_df.columns.get_level_values(0).unique()
for o in origins_list[:2]:
    origin_df_dict[italian_english_variables[o]] = get_average_swiss_df(origin_df[o],
        italian_english_variables, cantons_id)

Samples of the dataframes:

In [ ]:
origin_df_dict['Swiss'].set_index('id').head()

In [ ]:
origin_df_dict['Foreigners'].set_index('id').head()

We also want to obtain the difference in unemployment rate for foreigners and Swiss, for each canton:

In [ ]:
difference_origin_df = pd.DataFrame()
difference_origin_df['id'] = origin_df_dict['Foreigners']['id']
difference_origin_df['diff'] = (origin_df_dict['Foreigners']['Unemployment rate'] - 
    origin_df_dict['Swiss']['Unemployment rate'])

#### Create map to compare Swiss and foreign unemployment rates

In [ ]:
# Create a new map
map_swiss_origin = folium.Map(COORDINATE_SWITZERLAND,
    tiles='cartodbpositron', zoom_start=8)

# Plot unemployment rate for Swiss
map_swiss_origin.choropleth(geo_data=geo_json, name='Swiss',
    data=origin_df_dict['Swiss'], columns=['id', 'Unemployment rate'], key_on='feature.id', fill_opacity=1,
    fill_color='YlOrRd', topojson='objects.cantons')

# Plot unemployment rate for Foreigners
map_swiss_origin.choropleth(geo_data=geo_json, name='Foreign',
    data=origin_df_dict['Foreigners'], columns=['id', 'Unemployment rate'], key_on='feature.id', fill_opacity=1,
    fill_color='YlOrRd', topojson='objects.cantons')

# Plot difference in unemployment rates
map_swiss_origin.choropleth(geo_data=geo_json, name='Foreign - Swiss',
    data=difference_origin_df, columns=['id', 'diff'], key_on='feature.id', fill_opacity=1,
    fill_color='BuPu', topojson='objects.cantons')

# Add markers with information on unemployment per canton
#feature_group = FeatureGroup(name='Some icons')
#Marker(location=[45.3288, -121.6625],
       #popup='Mt. Hood Meadows').add_to(feature_group)

#Marker(location=[45.3311, -121.7113],
       #popup='Timberline Lodge').add_to(feature_group)

#feature_group.add_to(m)

# Add LayerControl
folium.LayerControl(collapsed=False).add_to(map_swiss_origin)

You can now interact with the map!
* You can look at the Swiss or Foreigners unemployment rates, or maybe plot the difference in both.
* The rate difference is always positive, because the unemployment rate is always higher for foreigners than for Swiss. It has been represented in a different color scale because it does not represent a *ratio*, but the *difference in ratios*. Cantons with a bigger difference in rates are represented in a darker color.

In [ ]:
map_swiss_origin

It is observed clearly that in most cantons the unemployment rate is higher for foreigners than for Swiss.

In most Switzerland there is a constant relatively low unemployment rate. However, it is higher in Ticino, Zürich and the French speaking part of Switzerland. 

As for the unemployment rate for Swiss people, it is higher in the West part of Switzerland. Jura and Neuchâtel have the highest unemployment rates.

TODO: Comment on the difference between the two rates.

## 3.2 Data per origin and age group

#### Compare unemployment rates including differences between age groups
Now we refine the analysis by adding the differences between age groups. Since it is nearly impossible to plot so many variables on a map we make a bar plot instead, which is a better option.

In [ ]:
# Read xls file containing unemployment data per age
age_df = pd.read_excel('{dp}{f}'.format(dp=DATA_PATH, f="ages {df}".format(df=AMSTAT_FILE)),
    header=[2,4,5])
age_df.drop(("Classi d'età 15-24, 15-49, 50 anni e più", 'Metriche', 'mese'), axis=1, inplace=True)

# We create a dictionary to map age class with the corresponding age range
age_labels_dic = {'1': '15-24', '2': '15-49', '3': '+50'}

# We create a dictionary of dataframes
age_df_dict = {}

# For each age we get the corresponding dataframe
ages_list = age_df.columns.get_level_values(0).unique()
for o in ages_list[:3]:
    age_df_dict[age_labels_dic[o]] = get_average_swiss_df(age_df[o],
        italian_english_variables, cantons_id)

Samples:

In [ ]:
age_df_dict['15-24'].set_index('id').head()

#### Get rate per age and origin

In [ ]:
def rename_row_values(df):
    nationality_rename_dic = {
        'stranieri':'foreign',
        'svizzeri':'swiss'
    }
    age_range_rename_dic = {
        '15-24 anni':'15-24',
        '25-49 anni':'25-49',
        '50 anni e più':'50+'
    }
    age_group_rename_dic = {
        '1':'1',
        '2':'2',
        '3':'3',
        'Totale':'Total'
    }
    renamed_df = df
    renamed_df['Nationality'] = renamed_df['Nationality'].apply(lambda x: nationality_rename_dic.get(x, 'all'))
    renamed_df['Age Range'] = renamed_df['Age Range'].apply(lambda x: age_range_rename_dic.get(x, float('NaN')))
    renamed_df['Age Group'] = renamed_df['Age Group'].apply(lambda x: age_group_rename_dic.get(x, 'Total'))
    return renamed_df

def refine_origin_age_df(df):
    refined_df = df
    refined_df.columns = df.columns.droplevel(1)
    refined_df.drop(['mese'], axis=1, inplace=True)
    rename_dic = {
        'Cantone':'Canton',
        'Nazionalità':'Nationality',
        'Classi d\'età 15-24, 15-49, 50 anni e più':'Age Group',
        'Ottobre 2016':'October 2016',
        'Novembre 2016':'November 2016',
        'Dicembre 2016':'December 2016',
        'Gennaio 2017':'January 2017',
        'Febbraio 2017':'February 2017',
        'Marzo 2017':'March 2017',
        'Aprile 2017':'April 2017',
        'Maggio 2017':'May 2017',
        'Giugno 2017':'June 2017',
        'Luglio 2017':'July 2017',
        'Agosto 2017':'August 2017',
        'Settembre 2017':'September 2017'
    }
    refined_df.rename(columns=rename_dic, inplace=True)
    refined_df.columns.values[2] = 'Age Range'
    refined_df = rename_row_values(refined_df)
    return refined_df

In [ ]:
origin_age_df = pd.read_excel('{dp}{f}'.format(dp=DATA_PATH, f=ORIGIN_AGE_FILE), header=[2, 3])

In [ ]:
origin_age_df = refine_origin_age_df(origin_age_df)
origin_age_df.set_index(origin_age_df['Age Group'], append=True, inplace=True)
origin_age_df.set_index(origin_age_df['Nationality'], append=True, inplace=True)

origin_age_df = origin_age_df.filter(like='20').mean(axis=1, skipna=True).to_frame()
origin_age_df.head(9)

In [ ]:
def get_origin_age_rates(df):
    index_value_dic = df.to_dict()[0]
    cantons = df.index.get_level_values(0).unique().tolist()
    cantons.remove('Totale')
    ages = range(1, 4)
    nationalities = ['foreign', 'swiss']
    
    rates_dic = {}
    swiss_population = index_value_dic[('Totale', 'Total', 'all')]
    for canton in cantons:
        canton_population = index_value_dic[(canton, 'Total', 'all')]
        for nationality in nationalities:
            nationality_population = index_value_dic[(canton, 'Total', nationality)]
            for age in ages:
                key = (canton, str(age), nationality)
                value = index_value_dic[key]
                rates_dic[key] = value / nationality_population
            rates_dic[(canton, 'Total', nationality)] = nationality_population / canton_population
        rates_dic[(canton, 'Total', 'all')] = canton_population / swiss_population
    
    rates_df = pd.DataFrame.from_dict(rates_dic, orient='index')
    rates_df.index = pd.MultiIndex.from_tuples(rates_df.index)
    rates_df.index.names = ['Canton', 'Age Group', 'Nationality']
    rates_df.columns = ['Rate']
    return rates_df

In [ ]:
origin_age_df = get_origin_age_rates(origin_age_df)
origin_age_df.head(9)

In [ ]:
by_canton_df = origin_age_df.reset_index()

# Slim down the Dataframe by selecting the needed rows
by_canton_df = by_canton_df.loc[ (by_canton_df['Age Group'] == 'Total') & (by_canton_df['Nationality'] == 'all' ) ]

# Keep the useful cols
by_canton_df.drop(['Age Group', 'Nationality'], axis=1, inplace=True)

# Index by canton name
by_canton_df.set_index('Canton', drop=True, inplace=True)

# Order by the rate
by_canton_df.sort_values(by='Rate', ascending=False, inplace=True)


# Show the plot
color = cm.inferno_r(np.linspace(.4,.8, 30))
by_canton_df.plot.bar(figsize=(17, 7), color=color)

plt.title('Ratio of ...')
plt.ylabel("Ratio")
plt.xlabel('Cantons')
plt.show()

TODO:

* Find a way to obtain the rate per age and origin!! (can't download dataset with both...). Maybe I misunderstood the wording?
* Barplot with the info. Suggestions:
    * Maybe draw it with horizontal bars (many cantons)
    * Group them by age
    * Overlap swiss and foreign
* Conclusions, colors, intervals...

# Task 4 <a name="t4">

We use the maps we have built for Switzerland to give a rough estimate of the difference in unemployment rates between the areas divided by the Röstigraben.

For that we will use the unemployment rates for Swiss people only, since we saw that in most cases the unemployment was higher for foreigners and therefore we can consider that the unemployment in an area might depend on the number of foreigners.

From what we saw [previously](#cm), we can set an unemployment rate of 4.2 as the limit between cantons in both sides of the Röstigraben.

In [ ]:
LIMIT_ROSTIGRABEN = 4.2

switzerland_df['class'] = (switzerland_df['Unemployment rate'] < LIMIT_ROSTIGRABEN)
switzerland_df['labels'] = switzerland_df['Unemployment rate'].apply(
    lambda x: 'German' if x < LIMIT_ROSTIGRABEN else 'French')

In [ ]:
# Create a new map
map_swiss = folium.Map(COORDINATE_SWITZERLAND,
    tiles='cartodbpositron', zoom_start=8)

# Create scale
minimum = min(switzerland_df['Unemployment rate'])
maximum = max(switzerland_df['Unemployment rate'])
rosti_scale = list(np.linspace(minimum, maximum, 2))

# Plot unemployment rate for Swiss
map_swiss.choropleth(geo_data=geo_json, name='Estimated Röstigraben',
    data=switzerland_df, columns=['id', 'class'], key_on='feature.id', fill_opacity=1,
    fill_color='Accent', topojson='objects.cantons', threshold_scale=rosti_scale)

map_swiss

To verify if their unemployment rates are very different or not, we compute the [harmonic mean](https://www.wikipedia.com/en/Harmonic_mean) for the two  groups of cantons that we obtained.

In [ ]:
swiss_mean = switzerland_df[['labels', 'Unemployment rate']].groupby('labels').agg(stats.hmean)
swiss_std = switzerland_df[['labels', 'Unemployment rate']].groupby('labels').agg(np.std)

In [ ]:
sns.set_style("whitegrid")

ind = np.arange(2)
barlist = plt.bar(ind, swiss_mean['Unemployment rate'], color='#d62728', yerr=swiss_std['Unemployment rate'])
barlist[0].set_color('#7fc97f')
barlist[1].set_color('#beaed4')

plt.ylabel('Mean unemployment rate')
plt.title('Unemployment rates in each side of the Röstigraben')
plt.xticks(ind, swiss_mean.index)

plt.show()

We can see that there is indeed a considerable difference in the unemployment rates for both groups of cantons.